In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [5]:
movies = pd.read_csv("./ml-1m/movies.dat",
                    sep="::",
                    header=None,
                    engine='python',
                    encoding='latin-1')

In [6]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
users =  pd.read_csv("./ml-1m/users.dat",
                    sep="::",
                    header=None,
                    engine='python',
                    encoding='latin-1')

In [9]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [10]:
ratings = pd.read_csv("./ml-1m/ratings.dat",
                    sep="::",
                    header=None,
                    engine='python',
                    encoding='latin-1')

In [11]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [16]:
training_set = pd.read_csv("./ml-100k/u1.base",
                          delimiter="\t")

In [17]:
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [ ]:
# user 1 rated movie 2, 3 stars

In [18]:
training_set = np.array(training_set,
                        dtype='int')

In [20]:
test_set = pd.read_csv("./ml-100k/u1.test",
                          delimiter="\t")
test_set = np.array(test_set,
                        dtype='int')

In [21]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]),max(test_set[:,1])))


In [22]:
# convert data into array with users in row movies in col
def convert(data):
    new_data = []
    for id_users in range(1,nb_users+1):
        id_movies = data[:,1][data[:,0]==id_users]
        id_ratings = data[:,2][data[:,0]==id_users] 
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [23]:
# converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)

In [24]:
test_set = torch.FloatTensor(test_set)

In [25]:
#convert the ratings into binary ratings 1 (liked) and 0 (not liked)
training_set[training_set == 0] = -1 
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0 
training_set[training_set >= 3] = 1

In [26]:
test_set[test_set == 0] = -1 
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0 
test_set[test_set >= 3] = 1

In [27]:
# creating the architecture of th NN
class RBM():
    def __init__(self, nv, nh): #num vis,hidden
        self.W = torch.randn(nh,nv)
        self.a = torch.randn(1,nh) # bias var for hidden
        self.b = torch.randn(1, nv) # bias for vis
        
    def sample_h(self, x): # prob of h given v
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    def sample_v(self, y): # prob of v given h
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    def train(self, v0, vk, ph0, phk): # contrastive divergence
        # v0 - input vector ratings by one user
        # vk - visible node after k iterations
        # ph0 - vector of probabilities that at first iteration hidden node=1 given values of v0
        # phk - prob of hidden nodes after k iteration given vk
        self.W += torch.mm(v0.t(),ph0) - torch.mm(vk.t(),phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
        

In [28]:
nv = len(training_set[0])
nh = 100 # trying out
batch_size = 100 # trying out
rbm = RBM(nv, nh)

In [30]:
# Training the RBM
nb_epoch = 10
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.0 # counter
    for id_user in range(0, nb_users -  batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0, _ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0] # freezing the unrated nodes
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s+=1.
    print("epoch: %s loss: %s"%(epoch,train_loss/s))

/usr/local/lib/python3.5/dist-packages/torch/tensor.py:297: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add_(other)


epoch: 1 loss: 0.2946010027100029
epoch: 2 loss: 0.24945888393547483
epoch: 3 loss: 0.2506743029328974
epoch: 4 loss: 0.24955058455215903
epoch: 5 loss: 0.25054960858427994
epoch: 6 loss: 0.24872581391629067
epoch: 7 loss: 0.25025089512977994
epoch: 8 loss: 0.2512377060201738
epoch: 9 loss: 0.24977862475836884
epoch: 10 loss: 0.251394184232132


In [31]:
# testing the RBM

In [32]:
test_loss = 0
s = 0.0 # counter
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1] # we are using activation of first training
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) >0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s+=1.
print("test loss: %s"%(test_loss/s))

test loss: 0.2430481043461903
